In [1]:
import numpy as np
import pandas as pd

import sys
sys.path.insert(0,'../')

import torch
import torch.nn as nn
from torch.optim import Adam
from torchtext import data
from torchtext import vocab
from torch.utils.tensorboard import SummaryWriter

from params import SEED
from models import RNN
import training_utils as tu
from pathlib import Path

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [2]:
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda')
torch.cuda.set_device(2)

In [3]:
TEXT = data.Field(stop_words=russian_stopwords, include_lengths=True)
LABEL = data.LabelField(use_vocab=False)
train_fields = {'category_id': ('label', LABEL), 'lemmatized': ('text', TEXT)}

data_path = '../data/json'

custom_embeddings = vocab.Vectors(name = '../embeddings/ft_native_300_ru_wiki_lenta_lemmatize.vec',
                                  cache = '../embeddings',
                                  unk_init = torch.Tensor.normal_)

train_data = data.TabularDataset(
    path=Path(data_path, f'dev.json'),
    format='json',
    fields=train_fields,
)

ID = data.Field(sequential=False, use_vocab=False)
test_fields = {'item_id': ('id', ID), 'lemmatized': ('text', TEXT)}
test_data = data.TabularDataset(
    path=Path(data_path, f'test.json'),
    format='json',
    fields=test_fields,
)

TEXT.build_vocab(train_data, vectors=custom_embeddings, unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

In [4]:
model_path = '../trained_models/final'
batch_size = 64
n_epochs = 10

input_dim = len(TEXT.vocab)
output_dim = len(LABEL.vocab)
embedding_dim = custom_embeddings.dim
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

hidden_dim = 200
n_layers = 3
bidirectional = True
dropout = 0.5

model = RNN(input_dim,
            output_dim,
            embedding_dim, 
            hidden_dim, 
            n_layers, 
            bidirectional,
            dropout, 
            pad_idx)

In [5]:
embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(embeddings)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

In [6]:
optimizer = Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)
model = model.to(device)

In [7]:
train_iterator = data.BucketIterator(train_data,
                                     batch_size=batch_size,
                                     sort_key=lambda ex:len(ex.text),
                                     sort_within_batch=True,
                                     device=device
                                    )

In [8]:
writer = SummaryWriter('../runs/final_test')

In [9]:
best, final = tu.train_model(model,
            train_iterator,
            train_iterator,
            optimizer, 
            criterion,
            model_path,
            n_epochs,
            '_',
            writer)

KeyboardInterrupt: 

In [31]:
model =model_cpu

In [32]:
model.load_state_dict(torch.load(model_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [33]:
device = torch.device('cuda')
torch.cuda.set_device(0)
model = model.to(device)

In [15]:
def predict(model, iterator):

    result = torch.LongTensor()
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text
            ids = batch.id.unsqueeze(1)

            predictions = model.forward(text, text_lengths).squeeze(1)
            predictions = predictions.argmax(dim=1, keepdim=True)
            batch_result = torch.cat([ids, predictions], dim=1).to('cpu')
            result = torch.cat([result, batch_result], dim=0)
    result = result.numpy()

    result = pd.DataFrame(result, columns=['item_id', 'category_id'])
    return result

In [28]:
test_iterator = data.BucketIterator(test_data,
                                     batch_size=batch_size*100,
                                     sort_key=lambda ex:len(ex.text),
                                     sort_within_batch=True,
                                     device=device
                                    )

In [34]:
predictions = predict(model, test_iterator)

RuntimeError: CUDA out of memory. Tried to allocate 7.54 GiB (GPU 0; 10.91 GiB total capacity; 8.56 GiB already allocated; 1.47 GiB free; 172.38 MiB cached)